In [5]:
import requests
from bs4 import BeautifulSoup

In [6]:
url = "https://www.fotmob.com/leagues/47/stats/season/20720/players/goals/premier-league-players"

In [7]:
def scrape_website(url, selector):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        section = soup.select_one(selector)
        if section:
            return section.prettify()
        else:
            return "Selector not found."
    else:
        return "Failed to fetch page."

In [8]:
response = scrape_website(url, '.css-11h0hl4-CardCSS.eenhk2w0')
print(response)

<div class="css-11h0hl4-CardCSS eenhk2w0">
 <style data-emotion="css 1yndnk3-LeagueSeasonStatsContainerCSS">
  .css-1yndnk3-LeagueSeasonStatsContainerCSS{display:-webkit-box;display:-webkit-flex;display:-ms-flexbox;display:flex;-webkit-flex-direction:column;-ms-flex-direction:column;flex-direction:column;}
 </style>
 <div class="css-1yndnk3-LeagueSeasonStatsContainerCSS e1vb2ffi4">
  <style data-emotion="css 1n72jcz-TLNavStickyCSS">
   .css-1n72jcz-TLNavStickyCSS{display:-webkit-box;display:-webkit-flex;display:-ms-flexbox;display:flex;-webkit-align-items:center;-webkit-box-align:center;-ms-flex-align:center;align-items:center;height:64px;padding:0 20px;-webkit-box-pack:justify;-webkit-justify-content:space-between;justify-content:space-between;position:-webkit-sticky;position:sticky;bottom:0;z-index:13;background-color:var(--GlobalColorScheme-Background-card);}.css-1n72jcz-TLNavStickyCSS button{display:-webkit-box;display:-webkit-flex;display:-ms-flexbox;display:flex;-webkit-align-ite